In [80]:
pip install azure-storage-blob

In [81]:
pip install snowflake-connector-python

In [82]:
#Import libraries
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine
import uuid

In [83]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()


In [84]:
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))

def get_week_of_year(date_str):
    """
    Calculate the ISO week number of the year for a given date.

    Parameters:
    date_str (str): A date string in the format 'YYYY-MM-DD'.

    Returns:
    int: ISO week number of the year.
    """
    # Parse the input string to a datetime object
    date = datetime.strptime(date_str, '%Y-%m-%d')

    # Get the ISO calendar week number
    week_of_year = date.isocalendar()[1]

    return week_of_year

In [85]:
# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Print the configuration
#Connection_STRING = config["connectionString"]

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'realestatesales'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)


real_est = pd.DataFrame()

# List all blobs in the specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = container_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))
    # Display the head of the DataFrame
    print(df.shape)
    # sind I have only one csv, I am doing to do the following instructions
    real_est = df.copy()


realestatesalescis9440.csv


<ipython-input-85-0290b84126e2>:30: DtypeWarning: Columns (16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(blob_content))


(1054159, 26)


In [86]:
real_est.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054159 entries, 0 to 1054158
Data columns (total 26 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   sid               1054159 non-null  object 
 1   id                1054159 non-null  object 
 2   position          1054159 non-null  int64  
 3   created_at        1054159 non-null  int64  
 4   created_meta      0 non-null        float64
 5   updated_at        1054159 non-null  int64  
 6   updated_meta      0 non-null        float64
 7   meta              1054159 non-null  object 
 8   Serial Number     1054159 non-null  int64  
 9   List Year         1054159 non-null  int64  
 10  Date Recorded     1054157 non-null  object 
 11  Town              1054159 non-null  object 
 12  Address           1054108 non-null  object 
 13  Assessed Value    1054159 non-null  int64  
 14  Sale Amount       1054159 non-null  float64
 15  Sales Ratio       1054159 non-null  float64
 16  

In [87]:
real_est.head()

,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,Serial Number,List Year,...,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,Counties,Zip Code,Town Index,Planning Regions
0,row-nct8~wv6h_sher,00000000-0000-0000-7C41-6C8A4092F57B,0,1666269278,NaN,1666269297,NaN,{ },2020177,2020,...,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014),1044.0,102.0,2.0,4.0
1,row-2y35-9qqi_7hpn,00000000-0000-0000-A402-71ABE687D103,0,1666269278,NaN,1666269297,NaN,{ },2020225,2020,...,Residential,Three Family,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,row-vd9u~7ew8-3ghk,00000000-0000-0000-1CA7-B48011B80E7F,0,1666269278,NaN,1666269297,NaN,{ },2020348,2020,...,Commercial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,row-g9u8-j5iy~38bh,00000000-0000-0000-44D9-FD3ABE7FAD53,0,1666269278,NaN,1666269297,NaN,{ },2020090,2020,...,Residential,Two Family,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,row-swq3~sutf_4mer,00000000-0000-0000-E5B6-CD8A9DCC0E5C,0,1666269278,NaN,1666269297,NaN,{ },200500,2020,...,Residential,Single Family,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
real_est_cleaned = real_est.copy()

In [89]:
real_est_cleaned.columns

Index(['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at',
       'updated_meta', 'meta', 'Serial Number', 'List Year', 'Date Recorded',
       'Town', 'Address', 'Assessed Value', 'Sale Amount', 'Sales Ratio',
       'Property Type', 'Residential Type', 'Non Use Code', 'Assessor Remarks',
       'OPM remarks', 'Location', 'Counties', 'Zip Code', 'Town Index',
       'Planning Regions'],
      dtype='object')

In [90]:
meta_data = ['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at', 'updated_meta', 'meta', 'Counties', 'Zip Code', 'Town Index',
       'Planning Regions']

In [91]:
real_est_cleaned.drop(columns=meta_data, inplace=True)

In [92]:
real_est_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054159 entries, 0 to 1054158
Data columns (total 14 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Serial Number     1054159 non-null  int64  
 1   List Year         1054159 non-null  int64  
 2   Date Recorded     1054157 non-null  object 
 3   Town              1054159 non-null  object 
 4   Address           1054108 non-null  object 
 5   Assessed Value    1054159 non-null  int64  
 6   Sale Amount       1054159 non-null  float64
 7   Sales Ratio       1054159 non-null  float64
 8   Property Type     671713 non-null   object 
 9   Residential Type  660275 non-null   object 
 10  Non Use Code      302242 non-null   object 
 11  Assessor Remarks  161472 non-null   object 
 12  OPM remarks       11564 non-null    object 
 13  Location          254643 non-null   object 
dtypes: float64(2), int64(3), object(9)
memory usage: 112.6+ MB


In [93]:
real_est_cleaned.head(10)

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020177,2020,2021-04-14T00:00:00,Ansonia,323 BEAVER ST,133000,248400.0,0.535400,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014)
1,2020225,2020,2021-05-26T00:00:00,Ansonia,152 JACKSON ST,110500,239900.0,0.460600,Residential,Three Family,NaN,NaN,NaN,NaN
2,2020348,2020,2021-09-13T00:00:00,Ansonia,230 WAKELEE AVE,150500,325000.0,0.463000,Commercial,NaN,NaN,NaN,NaN,NaN
3,2020090,2020,2020-12-14T00:00:00,Ansonia,57 PLATT ST,127400,202500.0,0.629100,Residential,Two Family,NaN,NaN,NaN,NaN
4,200500,2020,2021-09-07T00:00:00,Avon,245 NEW ROAD,217640,400000.0,0.544100,Residential,Single Family,NaN,NaN,NaN,NaN
5,200121,2020,2020-12-15T00:00:00,Avon,63 NORTHGATE,528490,775000.0,0.681900,Residential,Single Family,NaN,NaN,NaN,POINT (-72.89675 41.79445)
6,20058,2020,2021-06-01T00:00:00,Barkhamsted,46 RATLUM MTN RD,203530,415000.0,0.490434,Residential,Single Family,NaN,"2003 COLONIAL, 2140 SFLA, 2.99 AC",NaN,NaN
7,200046,2020,2021-01-25T00:00:00,Beacon Falls,34 LASKY ROAD,158030,243000.0,0.650300,Residential,Single Family,NaN,NaN,NaN,NaN
8,200016,2020,2020-11-13T00:00:00,Beacon Falls,9 AVON COURT,65590,100000.0,0.655900,Residential,Condo,NaN,NaN,NaN,NaN
9,2020360,2020,2021-08-10T00:00:00,Berlin,94 PERCIVAL AVE,140600,190790.0,0.736900,Residential,Single Family,NaN,NaN,NaN,NaN


In [94]:
real_est_cleaned.dtypes

Serial Number         int64
List Year             int64
Date Recorded        object
Town                 object
Address              object
Assessed Value        int64
Sale Amount         float64
Sales Ratio         float64
Property Type        object
Residential Type     object
Non Use Code         object
Assessor Remarks     object
OPM remarks          object
Location             object
dtype: object

In [95]:
real_est_cleaned.isnull().sum()

Serial Number             0
List Year                 0
Date Recorded             2
Town                      0
Address                  51
Assessed Value            0
Sale Amount               0
Sales Ratio               0
Property Type        382446
Residential Type     393884
Non Use Code         751917
Assessor Remarks     892687
OPM remarks         1042595
Location             799516
dtype: int64

In [96]:
#check for duplicates
serial_number_counts = real_est_cleaned['Serial Number'].value_counts()
print("Counts of each serial number:")
print(serial_number_counts)

Counts of each serial number:
Serial Number
10018        172
10002        172
10010        172
10015        171
10029        171
            ... 
1501564        1
1500492        1
1501533        1
1500584        1
210500004      1
Name: count, Length: 90873, dtype: int64


In [97]:
serial_number_10018 = real_est_cleaned[real_est_cleaned['Serial Number'] == 10018]
print(serial_number_10018.head(10))

       Serial Number  List Year        Date Recorded         Town  \
69032          10018       2001  2001-12-04T00:00:00      Ashford   
69956          10018       2001  2001-10-24T00:00:00      Ansonia   
70916          10018       2001  2002-01-17T00:00:00       Bozrah   
70981          10018       2001  2001-10-09T00:00:00   Manchester   
71631          10018       2001  2001-10-31T00:00:00  East Granby   
71902          10018       2001  2001-12-03T00:00:00       Bolton   
72028          10018       2001  2001-10-02T00:00:00       Hamden   
72938          10018       2001  2001-10-29T00:00:00     Brooklyn   
73196          10018       2001  2002-01-17T00:00:00      Hampton   
73413          10018       2001  2001-11-08T00:00:00   Burlington   

                  Address  Assessed Value  Sale Amount  Sales Ratio  \
69032  181 WESTFORD HL RD           77360     109822.0     0.704413   
69956   84-86 CLIFTON AVE           99190     127000.0     0.781024   
70916           BOZRAH ST  

In [98]:
num_duplicate_rows = real_est_cleaned.duplicated().sum()
print(num_duplicate_rows)
#no rows are duplicated

0


In [99]:
#drop columns 'Serial Number', 'Non Use Code', 'Assessor Remarks','OPM remarks', 'Location'
#drop NA for Address columns and Date Recorded
#most frequent for 'Property Type' and "Residential Type"

In [100]:
from sklearn.impute import SimpleImputer

In [101]:
imp_most_freq = SimpleImputer(strategy='most_frequent')

In [102]:
real_est_cleaned['Property Type'] = imp_most_freq.fit_transform(df['Property Type'].values.reshape(-1,1))[:,0]

In [103]:
real_est_cleaned['Residential Type'] = imp_most_freq.fit_transform(df['Residential Type'].values.reshape(-1,1))[:,0]

In [104]:
real_est_cleaned = real_est_cleaned.dropna(subset=['Address'])

In [105]:
real_est_cleaned = real_est_cleaned.dropna(subset=['Date Recorded'])

In [106]:
column_to_drop = ['Serial Number','Non Use Code', 'Assessor Remarks','OPM remarks', 'Location']

In [107]:
real_est_cleaned.drop(columns=column_to_drop, inplace=True)

In [108]:
real_est_cleaned.isnull().sum()

List Year           0
Date Recorded       0
Town                0
Address             0
Assessed Value      0
Sale Amount         0
Sales Ratio         0
Property Type       0
Residential Type    0
dtype: int64

In [109]:
real_est_cleaned.head(25)

,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
0,2020,2021-04-14T00:00:00,Ansonia,323 BEAVER ST,133000,248400.0,0.535400,Residential,Single Family
1,2020,2021-05-26T00:00:00,Ansonia,152 JACKSON ST,110500,239900.0,0.460600,Residential,Three Family
2,2020,2021-09-13T00:00:00,Ansonia,230 WAKELEE AVE,150500,325000.0,0.463000,Commercial,Single Family
3,2020,2020-12-14T00:00:00,Ansonia,57 PLATT ST,127400,202500.0,0.629100,Residential,Two Family
4,2020,2021-09-07T00:00:00,Avon,245 NEW ROAD,217640,400000.0,0.544100,Residential,Single Family
5,2020,2020-12-15T00:00:00,Avon,63 NORTHGATE,528490,775000.0,0.681900,Residential,Single Family
6,2020,2021-06-01T00:00:00,Barkhamsted,46 RATLUM MTN RD,203530,415000.0,0.490434,Residential,Single Family
7,2020,2021-01-25T00:00:00,Beacon Falls,34 LASKY ROAD,158030,243000.0,0.650300,Residential,Single Family
8,2020,2020-11-13T00:00:00,Beacon Falls,9 AVON COURT,65590,100000.0,0.655900,Residential,Condo
9,2020,2021-08-10T00:00:00,Berlin,94 PERCIVAL AVE,140600,190790.0,0.736900,Residential,Single Family


In [110]:
real_est_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1054108 entries, 0 to 1054158
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   List Year         1054108 non-null  int64  
 1   Date Recorded     1054108 non-null  object 
 2   Town              1054108 non-null  object 
 3   Address           1054108 non-null  object 
 4   Assessed Value    1054108 non-null  int64  
 5   Sale Amount       1054108 non-null  float64
 6   Sales Ratio       1054108 non-null  float64
 7   Property Type     1054108 non-null  object 
 8   Residential Type  1054108 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 80.4+ MB


In [111]:
num_duplicate_rows = real_est_cleaned.duplicated().sum()
print(num_duplicate_rows)

8728


In [112]:
real_est_cleaned.head(25)

,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
0,2020,2021-04-14T00:00:00,Ansonia,323 BEAVER ST,133000,248400.0,0.535400,Residential,Single Family
1,2020,2021-05-26T00:00:00,Ansonia,152 JACKSON ST,110500,239900.0,0.460600,Residential,Three Family
2,2020,2021-09-13T00:00:00,Ansonia,230 WAKELEE AVE,150500,325000.0,0.463000,Commercial,Single Family
3,2020,2020-12-14T00:00:00,Ansonia,57 PLATT ST,127400,202500.0,0.629100,Residential,Two Family
4,2020,2021-09-07T00:00:00,Avon,245 NEW ROAD,217640,400000.0,0.544100,Residential,Single Family
5,2020,2020-12-15T00:00:00,Avon,63 NORTHGATE,528490,775000.0,0.681900,Residential,Single Family
6,2020,2021-06-01T00:00:00,Barkhamsted,46 RATLUM MTN RD,203530,415000.0,0.490434,Residential,Single Family
7,2020,2021-01-25T00:00:00,Beacon Falls,34 LASKY ROAD,158030,243000.0,0.650300,Residential,Single Family
8,2020,2020-11-13T00:00:00,Beacon Falls,9 AVON COURT,65590,100000.0,0.655900,Residential,Condo
9,2020,2021-08-10T00:00:00,Berlin,94 PERCIVAL AVE,140600,190790.0,0.736900,Residential,Single Family


In [113]:
real_est_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1054108 entries, 0 to 1054158
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   List Year         1054108 non-null  int64  
 1   Date Recorded     1054108 non-null  object 
 2   Town              1054108 non-null  object 
 3   Address           1054108 non-null  object 
 4   Assessed Value    1054108 non-null  int64  
 5   Sale Amount       1054108 non-null  float64
 6   Sales Ratio       1054108 non-null  float64
 7   Property Type     1054108 non-null  object 
 8   Residential Type  1054108 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 80.4+ MB


In [114]:
num_duplicate_rows_Transact = real_est_cleaned.duplicated().sum()
print(num_duplicate_rows)

8728


In [115]:
real_est_cleaned.head(10)

,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
0,2020,2021-04-14T00:00:00,Ansonia,323 BEAVER ST,133000,248400.0,0.535400,Residential,Single Family
1,2020,2021-05-26T00:00:00,Ansonia,152 JACKSON ST,110500,239900.0,0.460600,Residential,Three Family
2,2020,2021-09-13T00:00:00,Ansonia,230 WAKELEE AVE,150500,325000.0,0.463000,Commercial,Single Family
3,2020,2020-12-14T00:00:00,Ansonia,57 PLATT ST,127400,202500.0,0.629100,Residential,Two Family
4,2020,2021-09-07T00:00:00,Avon,245 NEW ROAD,217640,400000.0,0.544100,Residential,Single Family
5,2020,2020-12-15T00:00:00,Avon,63 NORTHGATE,528490,775000.0,0.681900,Residential,Single Family
6,2020,2021-06-01T00:00:00,Barkhamsted,46 RATLUM MTN RD,203530,415000.0,0.490434,Residential,Single Family
7,2020,2021-01-25T00:00:00,Beacon Falls,34 LASKY ROAD,158030,243000.0,0.650300,Residential,Single Family
8,2020,2020-11-13T00:00:00,Beacon Falls,9 AVON COURT,65590,100000.0,0.655900,Residential,Condo
9,2020,2021-08-10T00:00:00,Berlin,94 PERCIVAL AVE,140600,190790.0,0.736900,Residential,Single Family


In [116]:
real_est_cleaned.isnull().sum()

List Year           0
Date Recorded       0
Town                0
Address             0
Assessed Value      0
Sale Amount         0
Sales Ratio         0
Property Type       0
Residential Type    0
dtype: int64

In [117]:
has_decimal = (real_est_cleaned['Sale Amount'].dropna() % 1 != 0).any()

print(f"Are there any values with digits after the decimal point? {has_decimal}")

Are there any values with digits after the decimal point? True


In [118]:
real_est_cleaned.head(10)

,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
0,2020,2021-04-14T00:00:00,Ansonia,323 BEAVER ST,133000,248400.0,0.535400,Residential,Single Family
1,2020,2021-05-26T00:00:00,Ansonia,152 JACKSON ST,110500,239900.0,0.460600,Residential,Three Family
2,2020,2021-09-13T00:00:00,Ansonia,230 WAKELEE AVE,150500,325000.0,0.463000,Commercial,Single Family
3,2020,2020-12-14T00:00:00,Ansonia,57 PLATT ST,127400,202500.0,0.629100,Residential,Two Family
4,2020,2021-09-07T00:00:00,Avon,245 NEW ROAD,217640,400000.0,0.544100,Residential,Single Family
5,2020,2020-12-15T00:00:00,Avon,63 NORTHGATE,528490,775000.0,0.681900,Residential,Single Family
6,2020,2021-06-01T00:00:00,Barkhamsted,46 RATLUM MTN RD,203530,415000.0,0.490434,Residential,Single Family
7,2020,2021-01-25T00:00:00,Beacon Falls,34 LASKY ROAD,158030,243000.0,0.650300,Residential,Single Family
8,2020,2020-11-13T00:00:00,Beacon Falls,9 AVON COURT,65590,100000.0,0.655900,Residential,Condo
9,2020,2021-08-10T00:00:00,Berlin,94 PERCIVAL AVE,140600,190790.0,0.736900,Residential,Single Family


In [119]:
unique_property_types = real_est_cleaned['Property Type'].unique()

print("Unique values in 'Property Type':")
print(unique_property_types)

Unique values in 'Property Type':
['Residential' 'Commercial' 'Vacant Land' 'Public Utility' 'Apartments'
 'Single Family' 'Industrial' 'Condo' 'Two Family' 'Three Family'
 'Four Family']


In [120]:
#creating dim_property_type dimention

#mapping dictionary
propertytype_mapping = {
    'Residential': 1,
    'Commercial': 2,
    'Vacant Land': 3,
    'Public Utility': 4,
    'Apartments': 5,
    'Single Family': 6,
    'Industrial': 7,
    'Condo': 8,
    'Two Family': 9,
    'Three Family': 10,
    'Four Family': 11
}

unique_propertytype = real_est_cleaned['Property Type'].unique()

# Converting the array of unique values into a DataFrame
unique_propertytype_df = pd.DataFrame(unique_propertytype, columns=['Property Type'])

# Applying the mapping to create a new column with descriptions
unique_propertytype_df['propertytype_id'] = unique_propertytype_df['Property Type'].map(propertytype_mapping)
unique_propertytype_df.rename(columns={'Property Type': 'propertytype'}, inplace=True)
unique_propertytype_df = unique_propertytype_df[unique_propertytype_df['propertytype'] != '<NA>']

unique_propertytype_df = unique_propertytype_df[['propertytype_id', 'propertytype']]

unique_propertytype_df

,propertytype_id,propertytype
0,1,Residential
1,2,Commercial
2,3,Vacant Land
3,4,Public Utility
4,5,Apartments
5,6,Single Family
6,7,Industrial
7,8,Condo
8,9,Two Family
9,10,Three Family


In [121]:
unique_residential_types = real_est_cleaned['Residential Type'].unique()

print("Unique values in 'Residential Type':")
print(unique_residential_types)


Unique values in 'Residential Type':
['Single Family' 'Three Family' 'Two Family' 'Condo' 'Four Family']


In [122]:
#creating dim_residential_type dimention

#mapping dictionary
residentialtype_mapping = {
    'Single Family': 1,
    'Three Family': 2,
    'Two Family': 3,
    'Condo': 4,
    'Four Family': 5
}
unique_residentialtype = real_est_cleaned['Residential Type'].unique()

# Converting the array of unique values into a DataFrame
unique_residentialtype_df = pd.DataFrame(unique_residentialtype, columns=['Residential Type'])

# Applying the mapping to create a new column with descriptions
unique_residentialtype_df['residentialtype_id'] = unique_residentialtype_df['Residential Type'].map(residentialtype_mapping)
unique_residentialtype_df.rename(columns={'Residential Type': 'residential_type'}, inplace=True)
unique_residentialtype_df = unique_residentialtype_df[unique_residentialtype_df['residential_type'] != '<NA>']

unique_residentialtype_df = unique_residentialtype_df[['residentialtype_id', 'residential_type']]

unique_residentialtype_df

,residentialtype_id,residential_type
0,1,Single Family
1,2,Three Family
2,3,Two Family
3,4,Condo
4,5,Four Family


In [123]:
# Rename the columns to uppercase
#unique_residentialtype_df.rename(columns={'residentialtype_id': 'RESIDENTIALTYPE_ID', 'residential_type': 'RESIDENTIAL_TYPE'}, inplace=True)


In [124]:
# creating dim_location dimension
unique_locations = real_est_cleaned[['Town', 'Address']].drop_duplicates()
unique_locations['location_id'] = range(1, len(unique_locations) + 1)
unique_locations.rename(columns={'Town': 'town', 'Address': 'address'}, inplace=True)
unique_locations = unique_locations[['location_id', 'town', 'address']]

In [125]:
print(unique_locations)

         location_id           town                    address
0                  1        Ansonia              323 BEAVER ST
1                  2        Ansonia             152 JACKSON ST
2                  3        Ansonia            230 WAKELEE AVE
3                  4        Ansonia                57 PLATT ST
4                  5           Avon               245 NEW ROAD
...              ...            ...                        ...
1054150       797579  West Hartford           97 MOHEGAN DRIVE
1054152       797580       Westport            140 EASTON ROAD
1054153       797581   Wethersfield              181 FOREST DR
1054155       797582       Woodbury  89 TAMARACK LANR UNIT 89A
1054156       797583       Woodbury         69 BACON POND ROAD

[797583 rows x 3 columns]


In [126]:
# Convert the 'Date Recorded' column to datetime
df['Date Recorded'] = pd.to_datetime(df['Date Recorded'], errors='coerce')

# Now find the minimum and maximum dates
start_date = df['Date Recorded'].min()
end_date = df['Date Recorded'].max()

In [127]:
start_date_y = df['List Year'].min()
end_date_y = df['List Year'].max()

In [128]:
print(start_date_y)
print(end_date_y)

2001
2021


In [129]:
print(start_date)

1999-04-05 00:00:00


In [130]:
print(end_date)

2022-09-30 00:00:00


In [131]:
# Function to calculate the week of the month for a given date
def week_of_month(dt):
    year = dt.year
    month = dt.month
    day = dt.day

    cal = calendar.monthcalendar(year, month)
    week_number = (day - 1) // 7 + 1
    return week_number

# Define start and end dates
start_date = pd.to_datetime('1999-04-05')
end_date = pd.to_datetime('2022-09-30')

# Create a DataFrame for the date dimension
date_dimension = pd.DataFrame({'date': pd.date_range(start_date, end_date, freq='D')})

# Extract attributes
date_dimension['year'] = date_dimension['date'].dt.year
date_dimension['quarter'] = date_dimension['date'].dt.quarter
date_dimension['month'] = date_dimension['date'].dt.month
date_dimension['monthname'] = date_dimension['date'].dt.strftime('%B')
date_dimension['day'] = date_dimension['date'].dt.day
date_dimension['dayname'] = date_dimension['date'].dt.strftime('%A')
date_dimension['date_iso_format'] = date_dimension['date'].apply(lambda x: x.date().isoformat())
date_dimension['date_id'] = date_dimension['date'].dt.strftime('%Y%m%d')
date_dimension['date_original_format'] = date_dimension['date'].dt.strftime('%Y-%m-%d')

# Add week of the month and week of the year
date_dimension['weekofMonth'] = date_dimension['date'].apply(week_of_month)
date_dimension['weekoftheyear'] = date_dimension['date'].dt.strftime('%U')

# Reorder columns
new_order = ['date_id', 'date_original_format', 'date_iso_format', 'year', 'quarter', 'month', 'day', 'monthname', 'dayname', 'weekoftheyear']
date_dimension = date_dimension[new_order]

print(date_dimension.head(25))

     date_id date_original_format date_iso_format  year  quarter  month  day  \
0   19990405           1999-04-05      1999-04-05  1999        2      4    5   
1   19990406           1999-04-06      1999-04-06  1999        2      4    6   
2   19990407           1999-04-07      1999-04-07  1999        2      4    7   
3   19990408           1999-04-08      1999-04-08  1999        2      4    8   
4   19990409           1999-04-09      1999-04-09  1999        2      4    9   
5   19990410           1999-04-10      1999-04-10  1999        2      4   10   
6   19990411           1999-04-11      1999-04-11  1999        2      4   11   
7   19990412           1999-04-12      1999-04-12  1999        2      4   12   
8   19990413           1999-04-13      1999-04-13  1999        2      4   13   
9   19990414           1999-04-14      1999-04-14  1999        2      4   14   
10  19990415           1999-04-15      1999-04-15  1999        2      4   15   
11  19990416           1999-04-16      1

In [132]:
date_dimension.tail(25)

,date_id,date_original_format,date_iso_format,year,quarter,month,day,monthname,dayname,weekoftheyear
8555,20220906,2022-09-06,2022-09-06,2022,3,9,6,September,Tuesday,36
8556,20220907,2022-09-07,2022-09-07,2022,3,9,7,September,Wednesday,36
8557,20220908,2022-09-08,2022-09-08,2022,3,9,8,September,Thursday,36
8558,20220909,2022-09-09,2022-09-09,2022,3,9,9,September,Friday,36
8559,20220910,2022-09-10,2022-09-10,2022,3,9,10,September,Saturday,36
8560,20220911,2022-09-11,2022-09-11,2022,3,9,11,September,Sunday,37
8561,20220912,2022-09-12,2022-09-12,2022,3,9,12,September,Monday,37
8562,20220913,2022-09-13,2022-09-13,2022,3,9,13,September,Tuesday,37
8563,20220914,2022-09-14,2022-09-14,2022,3,9,14,September,Wednesday,37
8564,20220915,2022-09-15,2022-09-15,2022,3,9,15,September,Thursday,37


In [133]:
real_est_cleaned.head()

,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
0,2020,2021-04-14T00:00:00,Ansonia,323 BEAVER ST,133000,248400.0,0.5354,Residential,Single Family
1,2020,2021-05-26T00:00:00,Ansonia,152 JACKSON ST,110500,239900.0,0.4606,Residential,Three Family
2,2020,2021-09-13T00:00:00,Ansonia,230 WAKELEE AVE,150500,325000.0,0.4630,Commercial,Single Family
3,2020,2020-12-14T00:00:00,Ansonia,57 PLATT ST,127400,202500.0,0.6291,Residential,Two Family
4,2020,2021-09-07T00:00:00,Avon,245 NEW ROAD,217640,400000.0,0.5441,Residential,Single Family


In [134]:
# Ensure 'Date Recorded' is in datetime format
real_est_cleaned['Date Recorded'] = pd.to_datetime(real_est_cleaned['Date Recorded'])

# Add a new column with the date in ISO format
real_est_cleaned['Date Recorded ISO'] = real_est_cleaned['Date Recorded'].dt.date.apply(lambda x: x.isoformat())

In [135]:
# Convert 'List Year' to string, add '-01-01', and then convert to datetime format
real_est_cleaned['List Year'] = pd.to_datetime(real_est_cleaned['List Year'].astype(str) + '-01-01')

In [136]:
# Assign 'residentialtype_id' to 'real_est_cleaned' DataFrame based on 'Residential Type'
real_est_cleaned['residentialtype_id'] = real_est_cleaned['Residential Type'].map(residentialtype_mapping)

In [137]:
# Assign 'propertytype_id' to 'real_est_cleaned' DataFrame based on 'Property Type'
real_est_cleaned['propertytype_id'] = real_est_cleaned['Property Type'].map(propertytype_mapping)

In [138]:
# Merge 'real_est_cleaned' with 'unique_locations' to assign 'location_id'
real_est_cleaned = pd.merge(real_est_cleaned, unique_locations, how='left', left_on=['Town', 'Address'], right_on=['town', 'address'])

# Drop the 'town' and 'address' columns as they are redundant
real_est_cleaned.drop(['town', 'address'], axis=1, inplace=True)

In [139]:
# Drop specified columns from 'real_est_cleaned' DataFrame
real_est_cleaned.drop(['Town', 'Address', 'Property Type', 'Residential Type'], axis=1, inplace=True)

In [140]:
real_est_cleaned.head()

,List Year,Date Recorded,Assessed Value,Sale Amount,Sales Ratio,Date Recorded ISO,residentialtype_id,propertytype_id,location_id
0,2020-01-01,2021-04-14,133000,248400.0,0.5354,2021-04-14,1,1,1
1,2020-01-01,2021-05-26,110500,239900.0,0.4606,2021-05-26,2,1,2
2,2020-01-01,2021-09-13,150500,325000.0,0.4630,2021-09-13,1,2,3
3,2020-01-01,2020-12-14,127400,202500.0,0.6291,2020-12-14,3,1,4
4,2020-01-01,2021-09-07,217640,400000.0,0.5441,2021-09-07,1,1,5


In [141]:
# Merge the two dataframes on 'Date Recorded ISO' in real_est_cleaned and 'date_iso_format' in date_dimension
merged_df = real_est_cleaned.merge(date_dimension, left_on='Date Recorded ISO', right_on='date_iso_format', how='left')

#Update the 'Date Recorded ISO' with 'date_id'
real_est_cleaned['Date Recorded ISO'] = merged_df['date_id']

# Verify the results
print(real_est_cleaned.head())

   List Year Date Recorded  Assessed Value  Sale Amount  Sales Ratio  \
0 2020-01-01    2021-04-14          133000     248400.0       0.5354   
1 2020-01-01    2021-05-26          110500     239900.0       0.4606   
2 2020-01-01    2021-09-13          150500     325000.0       0.4630   
3 2020-01-01    2020-12-14          127400     202500.0       0.6291   
4 2020-01-01    2021-09-07          217640     400000.0       0.5441   

  Date Recorded ISO  residentialtype_id  propertytype_id  location_id  
0          20210414                   1                1            1  
1          20210526                   2                1            2  
2          20210913                   1                2            3  
3          20201214                   3                1            4  
4          20210907                   1                1            5  


In [142]:
#date_dimension
date_dimension.head()

,date_id,date_original_format,date_iso_format,year,quarter,month,day,monthname,dayname,weekoftheyear
0,19990405,1999-04-05,1999-04-05,1999,2,4,5,April,Monday,14
1,19990406,1999-04-06,1999-04-06,1999,2,4,6,April,Tuesday,14
2,19990407,1999-04-07,1999-04-07,1999,2,4,7,April,Wednesday,14
3,19990408,1999-04-08,1999-04-08,1999,2,4,8,April,Thursday,14
4,19990409,1999-04-09,1999-04-09,1999,2,4,9,April,Friday,14


In [143]:
#loc dim
unique_locations.head()

,location_id,town,address
0,1,Ansonia,323 BEAVER ST
1,2,Ansonia,152 JACKSON ST
2,3,Ansonia,230 WAKELEE AVE
3,4,Ansonia,57 PLATT ST
4,5,Avon,245 NEW ROAD


In [144]:
#facts_realestate
real_est_cleaned.head()

,List Year,Date Recorded,Assessed Value,Sale Amount,Sales Ratio,Date Recorded ISO,residentialtype_id,propertytype_id,location_id
0,2020-01-01,2021-04-14,133000,248400.0,0.5354,20210414,1,1,1
1,2020-01-01,2021-05-26,110500,239900.0,0.4606,20210526,2,1,2
2,2020-01-01,2021-09-13,150500,325000.0,0.4630,20210913,1,2,3
3,2020-01-01,2020-12-14,127400,202500.0,0.6291,20201214,3,1,4
4,2020-01-01,2021-09-07,217640,400000.0,0.5441,20210907,1,1,5


In [145]:
# Convert both columns to string
real_est_cleaned['List Year'] = real_est_cleaned['List Year'].astype(str)
date_dimension['date_iso_format'] = date_dimension['date_iso_format'].astype(str)

# Merge the dataframes
merged_df = real_est_cleaned.merge(date_dimension, left_on='List Year', right_on='date_iso_format', how='left')

In [146]:
# Update the 'List Year' with 'date_id'
real_est_cleaned['List Year'] = merged_df['date_id']

In [147]:
real_est_cleaned.head()

,List Year,Date Recorded,Assessed Value,Sale Amount,Sales Ratio,Date Recorded ISO,residentialtype_id,propertytype_id,location_id
0,20200101,2021-04-14,133000,248400.0,0.5354,20210414,1,1,1
1,20200101,2021-05-26,110500,239900.0,0.4606,20210526,2,1,2
2,20200101,2021-09-13,150500,325000.0,0.4630,20210913,1,2,3
3,20200101,2020-12-14,127400,202500.0,0.6291,20201214,3,1,4
4,20200101,2021-09-07,217640,400000.0,0.5441,20210907,1,1,5


In [148]:
# Convert 'List Year' to datetime and format as 'date_id'
df['listeddate_id'] = pd.to_datetime(df['List Year'], format='%Y').dt.strftime('%Y%m%d')


new_column_names = {
    'Assessed Value': 'assessed_value',
    'Sale Amount': 'sales_amount',
    'Sales Ratio': 'sales_ratio',
    'location_id': 'location_id',
    'Date Recorded ISO': 'recordeddate_id',
    'List Year': 'listeddate_id',
}

real_est_cleaned = real_est_cleaned.rename(columns=new_column_names)

real_est_cleaned['fact_id'] = range(1, len(real_est_cleaned) + 1)

new_order = ['fact_id', 'sales_amount', 'assessed_value', 'sales_ratio', 'location_id','recordeddate_id', 'listeddate_id', 'propertytype_id', 'residentialtype_id']
real_est_cleaned = real_est_cleaned[new_order]

real_est_cleaned.head()



,fact_id,sales_amount,assessed_value,sales_ratio,location_id,recordeddate_id,listeddate_id,propertytype_id,residentialtype_id
0,1,248400.0,133000,0.5354,1,20210414,20200101,1,1
1,2,239900.0,110500,0.4606,2,20210526,20200101,1,2
2,3,325000.0,150500,0.4630,3,20210913,20200101,2,1
3,4,202500.0,127400,0.6291,4,20201214,20200101,1,3
4,5,400000.0,217640,0.5441,5,20210907,20200101,1,1


In [149]:
print(real_est_cleaned.columns)

Index(['fact_id', 'sales_amount', 'assessed_value', 'sales_ratio',
       'location_id', 'recordeddate_id', 'listeddate_id', 'propertytype_id',
       'residentialtype_id'],
      dtype='object')


In [150]:
real_est_cleaned.head(10)

,fact_id,sales_amount,assessed_value,sales_ratio,location_id,recordeddate_id,listeddate_id,propertytype_id,residentialtype_id
0,1,248400.0,133000,0.535400,1,20210414,20200101,1,1
1,2,239900.0,110500,0.460600,2,20210526,20200101,1,2
2,3,325000.0,150500,0.463000,3,20210913,20200101,2,1
3,4,202500.0,127400,0.629100,4,20201214,20200101,1,3
4,5,400000.0,217640,0.544100,5,20210907,20200101,1,1
5,6,775000.0,528490,0.681900,6,20201215,20200101,1,1
6,7,415000.0,203530,0.490434,7,20210601,20200101,1,1
7,8,243000.0,158030,0.650300,8,20210125,20200101,1,1
8,9,100000.0,65590,0.655900,9,20201113,20200101,1,4
9,10,190790.0,140600,0.736900,10,20210810,20200101,1,1


In [180]:
unique_residentialtype_df.head()

,residentialtype_id,residential_type
0,1,Single Family
1,2,Three Family
2,3,Two Family
3,4,Condo
4,5,Four Family


ValueError: time data "2021-04-14" doesn't match format "%d/%m/%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [181]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Update connection string information
host = "cis9440realestbaruch.postgres.database.azure.com"
dbname = "postgres"
user = "rabbybom"
password = "Flowerpot123!"
sslmode = "require"

# Construct connection string
conn_string = "postgresql+psycopg2://{0}:{1}@{2}/{3}?sslmode={4}".format(user, password, host, dbname, sslmode)
engine = create_engine(conn_string)

# Define your schema name
schema_name = "realest"

# Upload the DataFrame to PostgreSQL
unique_residentialtype_df.to_sql('dim_residential_type', engine, if_exists='append', index=False, schema=schema_name)

print("Data has been uploaded successfully")

Data has been uploaded successfully


In [186]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Update connection string information
host = "cis9440realestbaruch.postgres.database.azure.com"
dbname = "postgres"
user = "rabbybom"
password = "Flowerpot123!"
sslmode = "require"

# Construct connection string
conn_string = "postgresql+psycopg2://{0}:{1}@{2}/{3}?sslmode={4}".format(user, password, host, dbname, sslmode)
engine = create_engine(conn_string)

# Define your schema name
schema_name = "realest"

# Upload the DataFrame to PostgreSQL
unique_propertytype_df.to_sql('dim_property_type', engine, if_exists='append', index=False, schema=schema_name)

print("Data has been uploaded successfully")

Data has been uploaded successfully


In [191]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Update connection string information
host = "cis9440realestbaruch.postgres.database.azure.com"
dbname = "postgres"
user = "rabbybom"
password = "Flowerpot123!"
sslmode = "require"

# Construct connection string
conn_string = "postgresql+psycopg2://{0}:{1}@{2}/{3}?sslmode={4}".format(user, password, host, dbname, sslmode)
engine = create_engine(conn_string)

# Define your schema name
schema_name = "realest"

# Upload the DataFrame to PostgreSQL
unique_locations.to_sql('dim_location', engine, if_exists='append', index=False, schema=schema_name)

print("Data has been uploaded successfully")

Data has been uploaded successfully


In [151]:
real_est_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054108 entries, 0 to 1054107
Data columns (total 9 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   fact_id             1054108 non-null  int64  
 1   sales_amount        1054108 non-null  float64
 2   assessed_value      1054108 non-null  int64  
 3   sales_ratio         1054108 non-null  float64
 4   location_id         1054108 non-null  int64  
 5   recordeddate_id     1054108 non-null  object 
 6   listeddate_id       1054108 non-null  object 
 7   propertytype_id     1054108 non-null  int64  
 8   residentialtype_id  1054108 non-null  int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 72.4+ MB


         fact_id  sales_amount  assessed_value  sales_ratio  location_id  \
0              1      248400.0          133000       0.5354            1   
1              2      239900.0          110500       0.4606            2   
2              3      325000.0          150500       0.4630            3   
3              4      202500.0          127400       0.6291            4   
4              5      400000.0          217640       0.5441            5   
...          ...           ...             ...          ...          ...   
1054103  1054104      430000.0          263100       0.6118       661335   
1054104  1054105      200000.0           79810       0.3990       797582   
1054105  1054106      360000.0           79590       0.2210       797583   
1054106  1054107      275000.0          117600       0.4276       161794   
1054107  1054108      190000.0          130690       0.6878       685149   

        recordeddate_id listeddate_id  propertytype_id  residentialtype_id  
0         

In [79]:
real_est_cleaned.head()

,fact_id,sales_amount,assessed_value,sales_ratio,location_id,recordeddate_id,listeddate_id,propertytype_id,residentialtype_id
0,1,248400.0,133000,0.5354,1,20210414,20200101,1,1
2,3,325000.0,150500,0.4630,3,20210913,20200101,2,1
24,25,100000.0,47530,0.4753,25,20210115,20200101,3,1
92,93,315000.0,154210,0.4895,93,20210920,20200101,4,1
110,111,240000.0,120680,0.5028,111,20210528,20200101,5,1


In [152]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Update connection string information
host = "cis9440realestbaruch.postgres.database.azure.com"
dbname = "postgres"
user = "rabbybom"
password = "Flowerpot123!"
sslmode = "require"

# Construct connection string
conn_string = "postgresql+psycopg2://{0}:{1}@{2}/{3}?sslmode={4}".format(user, password, host, dbname, sslmode)
engine = create_engine(conn_string)

# Define your schema name
schema_name = "realest"

# Upload the DataFrame to PostgreSQL
real_est_cleaned.to_sql('facts_realestate', engine, if_exists='replace', index=False, schema=schema_name)

print("Data has been uploaded successfully")

Data has been uploaded successfully


In [447]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Update connection string information
host = "cis9440realestbaruch.postgres.database.azure.com"
dbname = "postgres"
user = "rabbybom"
password = "Flowerpot123!"
sslmode = "require"

# Construct connection string
conn_string = "postgresql+psycopg2://{0}:{1}@{2}/{3}?sslmode={4}".format(user, password, host, dbname, sslmode)
engine = create_engine(conn_string)

# Define your schema name
schema_name = "realest"

# Upload the DataFrame to PostgreSQL
date_dimension.to_sql('dim_date', engine, if_exists='append', index=False, schema=schema_name)

print("Data has been uploaded successfully")

Data has been uploaded successfully
